In [9]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import sqlalchemy

In [10]:
api_key = "93ba5366cb39b249dcbde95cc4d461e7b05b747d"
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []
#&ParameterName={parameter_name}
#url = (f"https://apps.bea.gov/api/data?&UserID={api_key}&method={get_data}&datasetname={dataset}&LineCode={line_code}&TableName={table_name}&GeoFIPS={geofip}&Year={year}")

#SAINC51
#response = requests.get(url).json()       
#list_response = requests.get(list_url).json()

In [38]:
total_trade = requests.get(f"{export_url}/hs?get=ALL_VAL_MO&key={api_key}&time=2019-01").json() 

In [39]:
total_trade[1][0]


'129545568405'

In [40]:
requests.get(f"https://api.census.gov/data/timeseries/intltrade/exports/hs?get=DF,ALL_VAL_MO,ALL_VAL_YR&YEAR=2013&MONTH=12").json()

[['DF', 'ALL_VAL_MO', 'ALL_VAL_YR', 'YEAR', 'MONTH'],
 ['-', '131915459503', '1578516879950', '2013', '12'],
 ['1', '114956497285', '1371134061205', '2013', '12'],
 ['2', '16958962218', '207382818745', '2013', '12']]

In [43]:
china_exports = requests.get(f"{export_url}/hs?get=ALL_VAL_MO,ALL_VAL_YR&key={api_key}&time=from+2015-01&CTY_CODE={china_code}").json()
pprint(china_exports)

[['ALL_VAL_MO', 'ALL_VAL_YR', 'time', 'CTY_CODE'],
 ['9459198394', '9459198394', '2015-01', '5700'],
 ['8754588178', '18213786572', '2015-02', '5700'],
 ['9886519164', '28100305736', '2015-03', '5700'],
 ['9279912850', '37380218586', '2015-04', '5700'],
 ['8749787563', '46130006149', '2015-05', '5700'],
 ['9615754115', '55745760264', '2015-06', '5700'],
 ['9505173559', '65250933823', '2015-07', '5700'],
 ['9183456081', '74434389904', '2015-08', '5700'],
 ['9419421166', '83853811070', '2015-09', '5700'],
 ['11324856248', '95178667318', '2015-10', '5700'],
 ['10603592681', '105782259999', '2015-11', '5700'],
 ['10091105315', '115873365314', '2015-12', '5700'],
 ['8202841436', '8202841436', '2016-01', '5700'],
 ['8070084594', '16272926030', '2016-02', '5700'],
 ['8916286008', '25189212038', '2016-03', '5700'],
 ['8672731969', '33861944007', '2016-04', '5700'],
 ['8539888162', '42401832169', '2016-05', '5700'],
 ['8838793248', '51240625417', '2016-06', '5700'],
 ['9128143401', '60368768818

In [44]:
requests.get(f"{export_url}/hs?get=CTY_CODE,CTY_NAME,ALL_VAL_MO,ALL_VAL_YR,E_COMMODITY_LDESC&key={api_key}&time=from+2015-01&CTY_CODE={china_code}").json()
print("clear")

clear


In [45]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&key={api_key}&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [47]:
len(export_by_commodity)

54073

In [48]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 54072:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [51]:

china_data.tail()

,YTD Value,Monthly Value,Period,Commodity,Comm Code
54066,320405,37821,2018-08,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620
54067,327518,7113,2018-09,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620
54068,361042,33524,2018-10,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620
54069,399482,38440,2018-11,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620
54070,444867,45385,2018-12,"MONOPODS, BIPODS, TRIPODS AND SIMILAR ARTICLES",9620


In [81]:
engine = create_engine('sqlite:///../data/china_trade.sqlite', echo=False)


In [2]:
#china_data.to_sql('export', con=engine, if_exists='replace') #dont run this again

In [24]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&key={api_key}&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()

In [55]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 55352:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
import_china_data = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [1]:
#import_china_data.to_sql('import', con=engine, if_exists='replace') 

In [ ]:
#import_china_data

In [11]:
engine = create_engine('sqlite:///../data/china_trade.sqlite')
#engine = create_engine("sqlite:///db/china_trade.sqlite")




In [12]:
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)


In [13]:
Base.classes.keys()

['export', 'imports']

In [14]:
Imports = Base.classes.imports

In [15]:
Exports = Base.classes.export

In [16]:
exports = session.query(Exports).first()
exports.__dict__


{'Comm Code': 305,
 'Commodity': 'FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL',
 'Monthly Value ': 274324,
 'Period': '2015-02',
 'YTD Value': 310025,
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1f2e9709470>,
 'index': 0}

In [23]:
results = session.query(Imports).statement


In [24]:
df = pd.read_sql_query(results, session.bind)

In [25]:
df

,index,YTD Value,Monthly Value,Period,Commodity,Comm Code
0,0,12238358,5385854,2015-02,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
1,1,16629923,4391565,2015-03,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
2,2,19522841,2892918,2015-04,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
3,3,23971868,4449027,2015-05,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
4,4,29755208,5783340,2015-06,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
5,5,35828070,6072862,2015-07,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
6,6,40715428,4887358,2015-08,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
7,7,46719724,6004296,2015-09,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
8,8,51674573,4954849,2015-10,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
9,9,57521143,5846570,2015-11,"FISH, DRIED, SALTED ETC, SMOKED ETC; ED FISH MEAL",305
